In [1]:
import json
import numpy as np
import os

cw = os.getcwd()
pwd = os.path.abspath(os.path.join(cw, "..", "results"))
print(f"Parent working directory: {pwd}")

Parent working directory: /proj/sourasb-220503/IoT_attack_CL_IDS/results


In [10]:
import os, json
import numpy as np

def compute_adaptation_rate(files):
    """
    Compute adaptation rate A(a_t) for a list of experiment result JSON files.
    
    Args:
        files (list[str]): List of file paths to JSON experiment result files.
    
    Returns:
        dict: Per-file statistics and overall summary.
    """
    per_file_stats = []
    per_file_avg_A = []
    per_file_std_A = []

    for file in files:
        with open(file, "r") as f:
            data = json.load(f)

        # --- Numerators: 2nd - 1st element from performance_m ---
        perf = data.get("performance_m", {}) or {}
        keys_in_order = list(perf.keys())
        numerators = []
        for k in keys_in_order:
            v = perf[k]
            if isinstance(v, (list, tuple)) and len(v) >= 2:
                numerators.append(float(v[0]) - float(v[1]))
            else:
                numerators.append(np.nan)

        # --- Denominators: TT from domain_training_cost dict (first number per key) ---
        cost_dict = data.get("domain_training_cost", {}) or {}
        denominators = []
        for k in keys_in_order:
            c = cost_dict.get(k, None)
            if isinstance(c, (list, tuple)) and len(c) >= 1:
                denominators.append(float(c[0]))
            else:
                denominators.append(np.nan)

        numerators = np.array(numerators, dtype=float)
        denominators = np.array(denominators, dtype=float)

        # Valid pairs: finite numerator and positive denominator
        valid = np.isfinite(numerators) & np.isfinite(denominators) & (denominators > 0)
        A_vals = numerators[valid] / denominators[valid]

        if A_vals.size:
            avg_num = float(np.mean(numerators[valid]))
            avg_A = float(np.mean(A_vals))
            std_A = float(np.std(A_vals))
            per_file_avg_A.append(avg_A)
            per_file_std_A.append(std_A)

            file_stats = {
                "file": os.path.basename(file),
                "domains_counted": int(A_vals.size),
                "total_domains": len(keys_in_order),
                "mean_numerator": avg_num,
                "mean_A": avg_A,
                "std_A": std_A,
            }
            per_file_stats.append(file_stats)

            print(f"{file_stats['file']}:")
            print(f"  Domains counted: {file_stats['domains_counted']}/{file_stats['total_domains']}")
            print(f"  Mean numerator (2nd - 1st perf): {file_stats['mean_numerator']:.6f}")
            print(f"  Mean adaptation rate: {file_stats['mean_A']:.6f}  (std: {file_stats['std_A']:.6f})")
        else:
            print(f"{os.path.basename(file)}: No valid adaptation-rate entries.")

    # --- Overall summary ---
    overall = {}
    if per_file_avg_A:
        overall["overall_mean_A"] = float(np.mean(per_file_avg_A))
        overall["overall_std_A"] = float(np.std(per_file_avg_A))

        print("\n===== Overall Adaptation Rate (per-file means) =====")
        print(f"Overall mean A: {overall['overall_mean_A']:.6f} ± {overall['overall_std_A']:.6f}")
    else:
        print("\nNo adaptation-rate statistics could be computed.")

    return {
        "per_file": per_file_stats,
        "overall": overall
    }


In [11]:
# List of files
files = [
    pwd + "/1_experiment_results_LSTM_WCL_random.json",
    pwd + "/2_experiment_results_LSTM_WCL_random.json",
    pwd + "/3_experiment_results_LSTM_WCL_random.json"
]

results = compute_adaptation_rate(files)


1_experiment_results_LSTM_WCL_random.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.248722
  Mean adaptation rate: -0.015381  (std: 0.016784)
2_experiment_results_LSTM_WCL_random.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.257262
  Mean adaptation rate: -0.017747  (std: 0.021387)
3_experiment_results_LSTM_WCL_random.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.257060
  Mean adaptation rate: -0.015724  (std: 0.016242)

===== Overall Adaptation Rate (per-file means) =====
Overall mean A: -0.016284 ± 0.001044


In [12]:
# List of files
files = [
    pwd + "/1_experiment_results_LSTM_WCL_b2w.json",
    pwd + "/2_experiment_results_LSTM_WCL_b2w.json",
    pwd + "/3_experiment_results_LSTM_WCL_b2w.json"
]

results = compute_adaptation_rate(files)


1_experiment_results_LSTM_WCL_b2w.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.122689
  Mean adaptation rate: -0.007935  (std: 0.009579)
2_experiment_results_LSTM_WCL_b2w.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.124748
  Mean adaptation rate: -0.008679  (std: 0.009980)
3_experiment_results_LSTM_WCL_b2w.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.140394
  Mean adaptation rate: -0.010042  (std: 0.012182)

===== Overall Adaptation Rate (per-file means) =====
Overall mean A: -0.008885 ± 0.000872


In [13]:
# List of files
files = [
    pwd + "/1_experiment_results_LSTM_WCL_w2b.json",
    pwd + "/2_experiment_results_LSTM_WCL_w2b.json",
    pwd + "/3_experiment_results_LSTM_WCL_w2b.json"
]

results = compute_adaptation_rate(files)


1_experiment_results_LSTM_WCL_w2b.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.419276
  Mean adaptation rate: -0.022095  (std: 0.014469)
2_experiment_results_LSTM_WCL_w2b.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.399570
  Mean adaptation rate: -0.020241  (std: 0.014231)
3_experiment_results_LSTM_WCL_w2b.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.388772
  Mean adaptation rate: -0.020466  (std: 0.014820)

===== Overall Adaptation Rate (per-file means) =====
Overall mean A: -0.020934 ± 0.000826


In [14]:
# List of files
files = [
    pwd + "/1_experiment_results_LSTM_WCL_toggle.json",
    pwd + "/2_experiment_results_LSTM_WCL_toggle.json",
    pwd + "/3_experiment_results_LSTM_WCL_toggle.json"
]

results = compute_adaptation_rate(files)


1_experiment_results_LSTM_WCL_toggle.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.308207
  Mean adaptation rate: -0.021611  (std: 0.019561)
2_experiment_results_LSTM_WCL_toggle.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.314007
  Mean adaptation rate: -0.022763  (std: 0.020621)
3_experiment_results_LSTM_WCL_toggle.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.322233
  Mean adaptation rate: -0.023740  (std: 0.022179)

===== Overall Adaptation Rate (per-file means) =====
Overall mean A: -0.022705 ± 0.000870


In [15]:
# List of files
files = [
    pwd + "/1_experiment_results_LSTM_SI_random.json",
    pwd + "/2_experiment_results_LSTM_SI_random.json",
    pwd + "/3_experiment_results_LSTM_SI_random.json"
]

results = compute_adaptation_rate(files)


1_experiment_results_LSTM_SI_random.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.024337
  Mean adaptation rate: -0.001418  (std: 0.003931)
2_experiment_results_LSTM_SI_random.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.027811
  Mean adaptation rate: -0.001656  (std: 0.004185)
3_experiment_results_LSTM_SI_random.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.032399
  Mean adaptation rate: -0.002059  (std: 0.004918)

===== Overall Adaptation Rate (per-file means) =====
Overall mean A: -0.001711 ± 0.000264


In [16]:
# List of files
files = [
    pwd + "/1_experiment_results_LSTM_SI_b2w.json",
    pwd + "/2_experiment_results_LSTM_SI_b2w.json",
    pwd + "/3_experiment_results_LSTM_SI_b2w.json"
]

results = compute_adaptation_rate(files)


1_experiment_results_LSTM_SI_b2w.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.030770
  Mean adaptation rate: -0.001420  (std: 0.002661)
2_experiment_results_LSTM_SI_b2w.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.032598
  Mean adaptation rate: -0.001622  (std: 0.002821)
3_experiment_results_LSTM_SI_b2w.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.025751
  Mean adaptation rate: -0.001051  (std: 0.001965)

===== Overall Adaptation Rate (per-file means) =====
Overall mean A: -0.001364 ± 0.000236


In [17]:
files = [    pwd + "/1_experiment_results_LSTM_SI_w2b.json",
    pwd + "/2_experiment_results_LSTM_SI_w2b.json",
    pwd + "/3_experiment_results_LSTM_SI_w2b.json"
]   
results = compute_adaptation_rate(files)

1_experiment_results_LSTM_SI_w2b.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.062905
  Mean adaptation rate: -0.004554  (std: 0.009514)
2_experiment_results_LSTM_SI_w2b.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.037931
  Mean adaptation rate: -0.002766  (std: 0.008831)
3_experiment_results_LSTM_SI_w2b.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.074155
  Mean adaptation rate: -0.005375  (std: 0.012305)

===== Overall Adaptation Rate (per-file means) =====
Overall mean A: -0.004232 ± 0.001089


In [19]:
files = [    pwd + "/1_experiment_results_LSTM_SI_toggle.json",
    pwd + "/2_experiment_results_LSTM_SI_toggle.json",
    pwd + "/3_experiment_results_LSTM_SI_toggle.json"
]
results = compute_adaptation_rate(files)

1_experiment_results_LSTM_SI_toggle.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.045009
  Mean adaptation rate: -0.003721  (std: 0.010533)
2_experiment_results_LSTM_SI_toggle.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.059214
  Mean adaptation rate: -0.003558  (std: 0.007085)
3_experiment_results_LSTM_SI_toggle.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.034628
  Mean adaptation rate: -0.002205  (std: 0.005899)

===== Overall Adaptation Rate (per-file means) =====
Overall mean A: -0.003161 ± 0.000680


In [20]:
# List of files
files = [
    pwd + "/1_experiment_results_LSTM_EWC_random.json",
    pwd + "/2_experiment_results_LSTM_EWC_random.json",
    pwd + "/3_experiment_results_LSTM_EWC_random.json"
]

results = compute_adaptation_rate(files)


1_experiment_results_LSTM_EWC_random.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.050463
  Mean adaptation rate: -0.000995  (std: 0.002176)
2_experiment_results_LSTM_EWC_random.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.111720
  Mean adaptation rate: -0.001789  (std: 0.003539)
3_experiment_results_LSTM_EWC_random.json:
  Domains counted: 47/48
  Mean numerator (2nd - 1st perf): -0.074252
  Mean adaptation rate: -0.002182  (std: 0.006395)

===== Overall Adaptation Rate (per-file means) =====
Overall mean A: -0.001656 ± 0.000494
